In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2

from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

In [11]:
df = pd.read_csv('data/baseline_model.csv')


In [12]:
df

,subscription_type,plan_type,auto_renew,avg_articles_per_week,days_since_last_login,support_tickets_last_90d,discount_used_last_renewal,email_open_rate,time_spent_per_session_mins,region,...,campaign_ctr,nps_score,sentiment_score,csat_score,customer_age,signup_source,payment_method,downgrade_history,subscription_status,tenure_days
0,Digital,Monthly,No,4.59,22,1,Yes,0.89,9.83,Europe,...,0.05,-19,0.29,5,58,Mobile App,Credit Card,No,Cancelled,1515
1,Digital+Print,Monthly,No,3.93,12,1,No,0.20,4.10,Asia,...,0.31,-94,-0.02,5,59,Mobile App,PayPal,No,Active,273
2,Digital,Annual,Yes,4.03,22,0,No,0.61,13.79,Europe,...,0.06,-24,-0.18,3,47,Mobile App,PayPal,Yes,Active,1534
3,Digital,Annual,No,0.95,17,0,No,0.09,10.22,Europe,...,0.35,21,0.36,2,22,Mobile App,PayPal,No,Cancelled,1705
4,Digital,Annual,No,4.82,27,0,No,0.39,11.69,Asia,...,0.26,-32,0.46,2,27,Mobile App,PayPal,No,Active,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Digital,Monthly,Yes,5.44,14,2,No,0.17,5.95,North America,...,0.63,-63,0.22,2,24,Referral,PayPal,No,Active,1355
996,Digital+Print,Annual,No,2.57,11,1,No,0.96,2.26,North America,...,0.17,-87,-0.21,4,24,Mobile App,Credit Card,No,Active,1548
997,Espresso,Monthly,Yes,5.39,9,0,Yes,0.85,7.80,Asia,...,0.43,10,0.66,2,66,Web,PayPal,Yes,Active,1686
998,Digital,Monthly,No,4.02,18,0,No,0.23,15.32,Asia,...,0.21,21,0.38,5,60,Mobile App,Credit Card,No,Cancelled,1095


In [56]:
df.columns

Index(['subscription_type', 'plan_type', 'auto_renew', 'avg_articles_per_week',
       'days_since_last_login', 'support_tickets_last_90d',
       'discount_used_last_renewal', 'email_open_rate',
       'time_spent_per_session_mins', 'completion_rate',
       'article_skips_per_week', 'previous_renewal_status', 'campaign_ctr',
       'nps_score', 'sentiment_score', 'csat_score', 'customer_age',
       'signup_source', 'downgrade_history', 'churn', 'tenure_days',
       'region_Asia', 'region_Europe', 'region_North America', 'region_Others',
       'most_read_Culture', 'most_read_Environment', 'most_read_Finance',
       'most_read_Politics', 'most_read_Technology', 'primary_device_Desktop',
       'primary_device_Mobile', 'primary_device_Tablet',
       'payment_method_Credit Card', 'payment_method_Debit Card',
       'payment_method_PayPal', 'last_campaign_engaged_Newsletter Promo',
       'last_campaign_engaged_Retention Offer',
       'last_campaign_engaged_Survey'],
      dtype='ob

In [61]:
df['csat_score'].max()

5

In [62]:
df['csat_score'].min()

1

In [13]:
df['subscription_type'] = df['subscription_type'].map({'Espresso':0,'Digital':1,'Digital+Print':2})


In [14]:
df['plan_type'] = df['plan_type'].map({'Monthly':0,'Annual':1})


In [15]:
df['auto_renew'] = df['auto_renew'].map({'Yes':1,'No':0})


In [16]:
df['discount_used_last_renewal'] = df['discount_used_last_renewal'].map({'Yes':1,'No':0})


In [17]:
df['downgrade_history'] = df['downgrade_history'].map({'Yes':1,'No':0})


In [18]:
df['previous_renewal_status'] = df['previous_renewal_status'].map({'Auto':1,'Manual':0})

In [19]:
df['signup_source'] = df['signup_source'].map({'Web':0,'Mobile App':0,'Referral':1})


In [20]:
df.rename(columns = {'subscription_status': 'churn'}, inplace = True)

In [21]:
df['churn'] = df['churn'].map({'Cancelled':1,'Active':0})

In [22]:
categorical_cols = df.select_dtypes(include=['object']).columns
categorical_cols

Index(['region', 'most_read_category', 'primary_device',
       'last_campaign_engaged', 'payment_method'],
      dtype='object')

In [103]:
df.columns

Index(['subscription_type', 'plan_type', 'auto_renew', 'avg_articles_per_week',
       'days_since_last_login', 'support_tickets_last_90d',
       'discount_used_last_renewal', 'email_open_rate',
       'time_spent_per_session_mins', 'completion_rate',
       'article_skips_per_week', 'previous_renewal_status', 'campaign_ctr',
       'nps_score', 'sentiment_score', 'csat_score', 'customer_age',
       'signup_source', 'downgrade_history', 'churn', 'tenure_days',
       'region_Asia', 'region_Europe', 'region_North America', 'region_Others',
       'most_read_Culture', 'most_read_Environment', 'most_read_Finance',
       'most_read_Politics', 'most_read_Technology', 'primary_device_Desktop',
       'primary_device_Mobile', 'primary_device_Tablet',
       'payment_method_Credit Card', 'payment_method_Debit Card',
       'payment_method_PayPal', 'last_campaign_engaged_Newsletter Promo',
       'last_campaign_engaged_Retention Offer',
       'last_campaign_engaged_Survey'],
      dtype='ob

In [24]:
df = pd.get_dummies(df, columns = ['region'], prefix = 'region')
df = pd.get_dummies(df, columns = ['most_read_category'], prefix = 'most_read')
df = pd.get_dummies(df, columns = ['primary_device'], prefix = 'primary_device')
df = pd.get_dummies(df, columns = ['payment_method'], prefix = 'payment_method')
df = pd.get_dummies(df, columns = ['last_campaign_engaged'], prefix = 'last_campaign_engaged')


In [25]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['churn']),
                                                 df['churn'],
                                                 test_size=0.2,
                                                random_state=42)

In [31]:
model_1 = LogisticRegression(class_weight='balanced')
model_1.fit(X_train, y_train)

c:\Users\adity\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

In [33]:
y_pred = model_1.predict(X_test)
accuracy_score(y_test, y_pred)

0.85

In [34]:
model_2 = XGBClassifier(enable_categorical = True, class_weight='balanced')
model_2.fit(X_train, y_train)

c:\Users\adity\Anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:03:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "class_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, ...)

In [35]:
y_pred = model_2.predict(X_test)
accuracy_score(y_test, y_pred)

0.97

In [36]:
model_3 = RandomForestClassifier(class_weight='balanced')
model_3.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced')

In [37]:
y_pred = model_3.predict(X_test)
accuracy_score(y_test, y_pred)

0.905

In [38]:
model_4 = DecisionTreeClassifier(class_weight='balanced')

In [39]:
model_4.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced')

In [40]:
y_pred = model_4.predict(X_test)
accuracy_score(y_test, y_pred)

0.955

In [41]:
import pickle

# Example: Assume `model` is your trained model
with open('model/model_1.pkl', 'wb') as f:
    pickle.dump(model_1, f)

In [42]:
with open('model/model_2.pkl', 'wb') as f:
    pickle.dump(model_2, f)

In [43]:
with open('model/model_3.pkl', 'wb') as f:
    pickle.dump(model_3, f)

In [44]:
with open('model/model_4.pkl', 'wb') as f:
    pickle.dump(model_4, f)

In [45]:
model_4.feature_names_in_

array(['subscription_type', 'plan_type', 'auto_renew',
       'avg_articles_per_week', 'days_since_last_login',
       'support_tickets_last_90d', 'discount_used_last_renewal',
       'email_open_rate', 'time_spent_per_session_mins',
       'completion_rate', 'article_skips_per_week',
       'previous_renewal_status', 'campaign_ctr', 'nps_score',
       'sentiment_score', 'csat_score', 'customer_age', 'signup_source',
       'downgrade_history', 'tenure_days', 'region_Asia', 'region_Europe',
       'region_North America', 'region_Others', 'most_read_Culture',
       'most_read_Environment', 'most_read_Finance', 'most_read_Politics',
       'most_read_Technology', 'primary_device_Desktop',
       'primary_device_Mobile', 'primary_device_Tablet',
       'payment_method_Credit Card', 'payment_method_Debit Card',
       'payment_method_PayPal', 'last_campaign_engaged_Newsletter Promo',
       'last_campaign_engaged_Retention Offer',
       'last_campaign_engaged_Survey'], dtype=object)